In [1]:
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import auc
import pandas as pd
from tqdm import tqdm
import os
import os
from sklearn.metrics import confusion_matrix
import torch
from eval_method_from_mtad import *

In [144]:
def adjust_anomaly_scores(scores, dataset, is_train, lookback):
    """
    Method for MSL and SMAP where channels have been concatenated as part of the preprocessing
    :param scores: anomaly_scores
    :param dataset: name of dataset
    :param is_train: if scores is from train set
    :param lookback: lookback (window size) used in model
    """

    # Remove errors for time steps when transition to new channel (as this will be impossible for model to predict)
    if dataset.upper() not in ['SMAP', 'MSL']:
        return scores

    adjusted_scores = scores.copy()
    if is_train:
        md = pd.read_csv(f'./mtad-gat-pytorch/datasets/data/{dataset.lower()}_train_md.csv')
    else:
        md = pd.read_csv('./mtad-gat-pytorch/datasets/data/labeled_anomalies.csv')
        md = md[md['spacecraft'] == dataset.upper()]

    md = md[md['chan_id'] != 'P-2']

    # Sort values by channel
    md = md.sort_values(by=['chan_id'])

    # Getting the cumulative start index for each channel
    sep_cuma = np.cumsum(md['num_values'].values) - lookback
    sep_cuma = sep_cuma[:-1]
    buffer = np.arange(1, 20)
    i_remov = np.sort(np.concatenate((sep_cuma, np.array([i+buffer for i in sep_cuma]).flatten(),
                                      np.array([i-buffer for i in sep_cuma]).flatten())))
    i_remov = i_remov[(i_remov < len(adjusted_scores)) & (i_remov >= 0)]
    i_remov = np.sort(np.unique(i_remov))
    if len(i_remov) != 0:
        adjusted_scores[i_remov] = 0

    # Normalize each concatenated part individually
    sep_cuma = np.cumsum(md['num_values'].values) - lookback
    s = [0] + sep_cuma.tolist()
    for c_start, c_end in [(s[i], s[i+1]) for i in range(len(s)-1)]:
        e_s = adjusted_scores[c_start: c_end+1]

        e_s = (e_s - np.min(e_s))/(np.max(e_s) - np.min(e_s))
        adjusted_scores[c_start: c_end+1] = e_s

    return adjusted_scores

In [716]:
# msl
# timestamp = "2022-09-20-23-39-53"
# dataset_name = "MSL"
# window_size = 128
# train_rec_error_list = np.load(f"/home/hehaowei/h0sda/battery_anomaly/spacecraft_save/msl/{timestamp}/feature/train.npy")
# rec_error_list = np.load(f"/home/hehaowei/h0sda/battery_anomaly/spacecraft_save/msl/{timestamp}/mean/test.npy")
# label = np.load("/home/hehaowei/h0sda/battery_anomaly/OmniAnomaly/processed/MSL_test_label.pkl", allow_pickle=True)[window_size:]

# smap
timestamp = '2022-09-20-22-55-02'
dataset_name = "SMAP"
window_size = 100
train_rec_error_list = np.load(f"/home/hehaowei/h0sda/battery_anomaly/spacecraft_save/smap/{timestamp}/feature/train.npy")
rec_error_list = np.load(f"/home/hehaowei/h0sda/battery_anomaly/spacecraft_save/smap/{timestamp}/mean/test.npy")
label = np.load("/home/hehaowei/h0sda/battery_anomaly/OmniAnomaly/processed/SMAP_test_label.pkl", allow_pickle=True)[window_size:]

print(train_rec_error_list.shape, rec_error_list.shape, label.shape)

(135083,) (427517,) (427517,)


In [711]:
train_rec_error_list_adjusted = adjust_anomaly_scores(train_rec_error_list, dataset_name, True, window_size)
rec_error_list_adjusted = adjust_anomaly_scores(rec_error_list, dataset_name, False, window_size)
print(train_rec_error_list_adjusted.shape)

(135083,)


In [712]:
e_eval = epsilon_eval(train_rec_error_list_adjusted, rec_error_list_adjusted, label, reg_level=0)
print(e_eval)

best_epsilon 0.1647522710263729
{'f1': 0.9136677738625707, 'precision': 0.8882289974641364, 'recall': 0.940617229607171, 'TP': 51448, 'TN': 366347, 'FP': 6474, 'FN': 3248, 'threshold': 0.1647522710263729, 'latency': 123.19678303517722, 'reg_level': 0}


In [714]:
bf_eval = bf_search(rec_error_list_adjusted, label, start=0.2, end=0.4, step_num=10, verbose=True)
print(bf_eval)

Finding best f1-score by searching for threshold..
search range:  0.2 0.4
0
cur thr:  0.22 (0.914466937547656, 0.9157545421198403, 0.9131929207047694, 49948, 368226, 4595, 4748) (0.914466937547656, 0.9157545421198403, 0.9131929207047694, 49948, 368226, 4595, 4748) 0.22
1
cur thr:  0.24 (0.9188408319650762, 0.9245691648141404, 0.9131929207047694, 49948, 368746, 4075, 4748) (0.9188408319650762, 0.9245691648141404, 0.9131929207047694, 49948, 368746, 4075, 4748) 0.24
2
cur thr:  0.26 (0.9237434053993261, 0.9345507613414379, 0.9131929207047694, 49948, 369323, 3498, 4748) (0.9237434053993261, 0.9345507613414379, 0.9131929207047694, 49948, 369323, 3498, 4748) 0.26
3
cur thr:  0.28 (0.9283188403111698, 0.9439646209922014, 0.9131929207047694, 49948, 369856, 2965, 4748) (0.9283188403111698, 0.9439646209922014, 0.9131929207047694, 49948, 369856, 2965, 4748) 0.28
4
cur thr:  0.30000000000000004 (0.9312873031989332, 0.9501236444835222, 0.9131929207047694, 49948, 370199, 2622, 4748) (0.9312873031989